## Reference
- [TensorFlow Example - convolution network](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3%20-%20Neural%20Networks/convolutional_network.ipynb)

In [1]:
import tensorflow as tf
import numpy as np
import input_data

mnist = input_data.read_data_sets("./../data/MNIST/", one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/Dongjun/DLselfProjects/4.Convolution_Network/input_data.py:41: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting ./../data/MNIST/train-images-idx3-ubyte.gz
Extracting ./../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ./../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ./../data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
num_iters = 1000
batch_size = 128

geometry = [28, 28]
classes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
num_classes = len(classes)
dropout_prob = 0.5

In [4]:
# Tensor Flow Graph Input
X = tf.placeholder(tf.float32, [None, geometry[0]*geometry[1]])
y = tf.placeholder(tf.float32, [None, num_classes])
dropout = tf.placeholder(tf.float32)

# Weight & bias
# 5x5 conv, 1 input, 32 outputs
Wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

# 5x5 conv, 32 input, 64 outputs
Wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

# Fully connected (Standard 2-layer MLP), 7*7*64 input, 1024 
Wf1 = tf.Variable(tf.random_normal([7*7*64, 1024]))
bf1 = tf.Variable(tf.random_normal([1024]))

Wf2 = tf.Variable(tf.random_normal([1024, num_classes]))
bf2 = tf.Variable(tf.random_normal([num_classes]))

In [5]:
# Convolution Network

# Reshape input picture
input_X = tf.reshape(X, shape=[-1, 28, 28, 1])

# Stage 1 : Convolution -> ReLU -> Max Pooling -> Dropout
conv1 = tf.nn.conv2d(input_X, Wc1, strides = [1, 1, 1, 1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, bc1))
conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')
conv1 = tf.nn.dropout(conv1, dropout)

# Stage 2 : Convolution -> ReLU -> Max Pooling -> Dropout
conv2 = tf.nn.conv2d(conv1, Wc2, strides = [1, 1, 1, 1], padding='SAME')
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, bc2))
conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')
conv2 = tf.nn.dropout(conv2, dropout)

# Stage 3 : Fully connected : Linear -> ReLU -> Dropout
fc1 = tf.reshape(conv2, [-1, Wf1.get_shape().as_list()[0]])
fc1 = tf.nn.relu(tf.add(tf.matmul(fc1, Wf1), bf1))
fc1 = tf.nn.dropout(fc1, dropout)

fc2 = tf.add(tf.matmul(fc1, Wf2), bf2)

In [6]:
# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc2, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
init = tf.initialize_all_variables()

# Launch the Graph
with tf.Session() as sess:
    sess.run(init)
    
    # Train
    for epoch in range(1, num_iters+1):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training data
        sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout: dropout_prob})
        
        if epoch & 50 == 0:
            loss = sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout: 1.})
            print("Epoch : ", epoch, " loss=" , loss)
    
    print("Optimization Finishied")
    
    # Test
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                             y: mnist.test.labels, 
                                                             dropout: 1.}) )

Epoch :  1  loss= 104725.0
Epoch :  4  loss= 59721.9
Epoch :  5  loss= 49071.1
Epoch :  8  loss= 32064.8
Epoch :  9  loss= 33889.2
Epoch :  12  loss= 27574.1
Epoch :  13  loss= 26917.3
Epoch :  64  loss= 9776.74
Epoch :  65  loss= 9796.68
Epoch :  68  loss= 9503.35
Epoch :  69  loss= 9953.09
Epoch :  72  loss= 12185.1
Epoch :  73  loss= 8597.04
Epoch :  76  loss= 14026.4
Epoch :  77  loss= 12563.6
Epoch :  128  loss= 5334.76
Epoch :  129  loss= 6143.45
Epoch :  132  loss= 5221.77
Epoch :  133  loss= 4630.61
Epoch :  136  loss= 4281.14
Epoch :  137  loss= 5493.57
Epoch :  140  loss= 5504.78
Epoch :  141  loss= 3767.55
Epoch :  192  loss= 4144.98
Epoch :  193  loss= 3370.35
Epoch :  196  loss= 3394.21
Epoch :  197  loss= 3557.54
Epoch :  200  loss= 3972.59
Epoch :  201  loss= 3087.53
Epoch :  204  loss= 3358.9
Epoch :  205  loss= 2432.73
Epoch :  256  loss= 1662.3
Epoch :  257  loss= 2930.95
Epoch :  260  loss= 2977.37
Epoch :  261  loss= 2926.36
Epoch :  264  loss= 1522.62
Epoch :  265 